In [ ]:
import os
print(os.getcwd())

from build_model import *

In [ ]:
args = generic.init_args()
args["model_path"] = "models/rnn_market_model.pt"
args["gen_path"] = "models/rnn_gen_model.pt"
args["num_gpus"] = 0

args["epochs"] = 2  # debug: 2, normal: 100

with open("data/data.pkl", "rb") as file:
    data = pickle.load(file)

init_date = pd.to_datetime("20140101", format="%Y%m%d")
fin_date = pd.to_datetime("20160531", format="%Y%m%d")
# 20160601 - 20161231 is left out for testing

# Change ind_foc industry codes to include different industries here
od, date_count = load_data(
    data, init_date, fin_date, 
    ind_foc=("itech", )
)

sel_ret = load_market_data('data/ff5_daily_clean.csv')
documents, ky_ret, ky_dates = prepare_data(
    data, sel_ret, date_count
)

word2embedding, embedding_dim = load_glove_embedding(
    'data/glove.6B.100d.txt'
)
preprocess_documents(documents, word2embedding)

word_thres = 10
vocab = LanguageIndex(documents, word_thres)
pretrained_embedding = utils.get_pretained_glove(
    vocab.word2idx, word2embedding, embedding_dim
)
pretrained_embedding = pretrained_embedding.transpose((1,0))

x_tr, m_tr, y_tr, x_te, m_te, y_te = load_train_test(
    documents, ky_ret, vocab
)

train_dataset = rnnDataset(x_tr, y_tr, m_tr)
val_dataset = rnnDataset(x_te, y_te, m_te)

model = encoder.Encoder(pretrained_embedding, args)
gen = generator.Generator(pretrained_embedding, args)

In [ ]:
epoch_stats, model, gen = train.train_model(
    train_dataset, val_dataset, model, gen, args
)

In [ ]:
torch.save(model.state_dict(), args["model_path"])
torch.save(gen.state_dict(), args["gen_path"])